In [1]:
import segmentation_models as sm

sm.set_framework('tf.keras')

from cnn.input import PascalVOC2012Dataset, Dataloader, get_training_augmentation

import os

2022-07-07 22:44:37.493774: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-07 22:44:37.493791: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Segmentation Models: using `keras` framework.


In [2]:
train_dataset_descriptor_filepath = os.path.join(
    'pascalvoc12',
    'VOCdevkit',
    'VOC2012',
    'ImageSets',
    'Segmentation',
    'train.txt',
)

val_dataset_descriptor_filepath = os.path.join(
    'pascalvoc12',
    'VOCdevkit',
    'VOC2012',
    'ImageSets',
    'Segmentation',
    'val.txt',
)

images_path = os.path.join(
    'pascalvoc12', 
    'VOCdevkit', 
    'VOC2012', 
    'JPEGImages'
)

masks_path = os.path.join(
    'pascalvoc12', 
    'VOCdevkit', 
    'VOC2012', 
    'SegmentationClass'
) 

In [3]:
image_height = 128
image_width = 128

In [4]:
train_dataset = PascalVOC2012Dataset(
    train_dataset_descriptor_filepath,
    images_path=images_path,
    masks_path=masks_path,
    image_height=image_height,
    image_width=image_width,
    augmentation=get_training_augmentation(image_height, image_width),
)

val_dataset = PascalVOC2012Dataset(
    val_dataset_descriptor_filepath,
    images_path=images_path,
    masks_path=masks_path,
    image_height=image_height,
    image_width=image_width,
)

/home/guilherme/git/segqnas/.venv/lib/python3.8/site-packages/albumentations/augmentations/transforms.py:2029: UserWarning: Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().
  warnings.warn('Using lambda is incompatible with multiprocessing. '


In [7]:
train_dataloader = Dataloader(train_dataset, batch_size=64, shuffle=True)
val_dataloader = Dataloader(val_dataset, batch_size=64, shuffle=False)

In [8]:
net = sm.Unet('resnet34', classes=21, input_shape=(image_height, image_width, 3), activation='softmax', encoder_weights='imagenet')

net.compile(                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
    optimizer='Adam',
    loss=sm.losses.categorical_focal_dice_loss,
    metrics=[sm.metrics.IOUScore(threshold=0.5)],
)

history = net.fit(train_dataloader, 
                validation_data=val_dataloader,
                epochs=50)

Epoch 1/50
22/22 [==============================] - 160s 7s/step - loss: 0.9805 - iou_score: 0.0352 - val_loss: 1.0058 - val_iou_score: 0.0363
Epoch 2/50
 7/22 [========>.....................] - ETA: 1:25 - loss: 0.9530 - iou_score: 0.0706

KeyboardInterrupt: 